In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
questions = []
data_path = "/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/zac2021-ltr-data/train_question_answer.json"
with open(data_path) as json_file:
    questions_json = json.load(json_file)
    questions = questions_json['items']
# for question in questions_json['items']:
#     label = question["relevant_articles"][0]['law_id'] + "###" + question["relevant_articles"][0]['article_id']
#     q = question['question']
#     id = question['question_id']
#     questions.append([id, q, label])

In [ ]:
questions[0]

{'question': 'Công an xã xử phạt lỗi không mang bằng lái xe có đúng không?',
 'question_id': '0637bf82c8b290c7875c5bfddbf91df5',
 'relevant_articles': [{'article_id': '7', 'law_id': '47/2011/tt-bca'}]}

## Chia train test từ file Question.json


In [ ]:
questions_train = questions[:int(0.85*len(questions))]
questions_test = questions[int(0.85*len(questions)) : ]
print("Số câu hỏi tập train: ", len(questions_train))
print("Số câu hỏi tập test: ", len(questions_test))

Số câu hỏi tập train:  2716
Số câu hỏi tập test:  480


In [ ]:
questions_train[2715]

{'question': 'Thời hạn hiệu lực quyết định công bố hoạt động bến thủy nội địa phục vụ thi công công trình chính được quy định như thế nào?',
 'question_id': '181790aebab8ff9a9ce377a90fe0527c',
 'relevant_articles': [{'article_id': '19', 'law_id': '08/2021/nđ-cp'}]}

In [ ]:
questions_test[0]

{'question': 'Những dịch vụ nào được cung cấp trực tuyến trong việc kiểm soát hộ chiếu công dân Việt Nam?',
 'question_id': '142e0483fd375f557561ace15e2aea83',
 'relevant_articles': [{'article_id': '8', 'law_id': '77/2020/nđ-cp'}]}

In [ ]:
import codecs
import os
dest_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data'
file_name = 'questions_train.json'

file_path = os.path.join(dest_path, file_name)
with codecs.open(file_path, "w", encoding='utf8') as outfile:
        json.dump(questions_train , outfile, ensure_ascii=False)

In [ ]:
import codecs
import os
dest_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data'
file_name = 'questions_test.json'

file_path = os.path.join(dest_path, file_name)
with codecs.open(file_path, "w", encoding='utf8') as outfile:
        json.dump(questions_test , outfile, ensure_ascii=False)

## Các functions xử lý

In [ ]:
!pip install underthesea

     |████████████████████████████████| 7.6 MB 12.2 MB/s 
     |████████████████████████████████| 235 kB 61.1 MB/s 
     |████████████████████████████████| 965 kB 63.1 MB/s 
     |████████████████████████████████| 581 kB 57.9 MB/s 


In [ ]:
import string
from underthesea import word_tokenize
import os
import json

In [ ]:
# Tiến xử lý văn bản
from underthesea import word_tokenize
def remove_stopword(w):
    stop_word = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10" ,
            "của", "và", "các", "có", "được", "theo", "tại", "trong", "về", 
            "hoặc", "người",  "này", "khoản", "cho", "không", "từ", "phải", 
            "ngày", "việc", "sau",  "để",  "đến", "bộ",  "với", "là", "năm", 
            "khi", "số", "trên", "khác", "đã", "thì", "thuộc", "điểm", "đồng",
            "do", "một", "bị", "vào", "lại", "ở", "nếu", "làm", "đây", 
            "như", "đó", "mà", "nơi", "”", "“"]
    return w not in stop_word

def remove_punctuation(w):
    return w not in string.punctuation

def lower_case(w):
    return w.lower()

In [ ]:
# BM25 Tokenizer
def bm25_tokenizer(text):
    tokens = word_tokenize(text)
    tokens = list(map(lower_case, tokens))
    tokens = list(filter(remove_punctuation, tokens))
    tokens = list(filter(remove_stopword, tokens))
    return tokens

In [ ]:
# Chỉ số F2
def calculate_f2(precision, recall):        
    return (5 * precision * recall) / (4 * precision + recall + 1e-20)

## BM25 train

In [ ]:
!pip install rank_bm25

In [ ]:
import os
import json
import pickle
import numpy as np
from tqdm import tqdm
from rank_bm25 import *
dest_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data'
legal_corpus_path = "/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/zac2021-ltr-data/legal_corpus.json"
model_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Model'

In [ ]:
# Import data văn bản pháp luật

legal_corpus_path = "/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/zac2021-ltr-data/legal_corpus.json"
with open(legal_corpus_path) as json_file:
    data = json.load(json_file)
print(data[0])
print(type(data))

{'law_id': '01/2009/tt-bnn', 'articles': [{'article_id': '1', 'title': 'Điều 1. Phạm vi áp dụng', 'text': 'Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.'}, {'article_id': '2', 'title': 'Điều 2. Tổ chức lực lượng', 'text': '1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân cấp xã nơi có đê phải tổ chức lực lượng lao động tại địa phương để tuần tra, canh gác đê và thường trực trên các điếm canh đê hoặc nhà dân khu vực gần đê (đối với những khu vực chưa có điếm canh đê), khi có báo động lũ từ cấp I trở lên đối với tuyến sông có đê (sau đây gọi tắt là lực lượng tuần tra, canh gác đê).\n2. Lực lượng tuần tra, canh gác đê được tổ chức thành các đội, do Ủy ban nhân dân cấp xã ra quyết định thành lập; từ 01 đến 02 kilômét đê thành lập 01 đội; mỗi đội có từ 12 đến 18 người, trong đó có 01 đội trưởng và 01 hoặc 02 đội phó. Danh sách thành viên đội tuần tra, canh gác đê được niêm yết

In [ ]:
# Tạo file legal_dict
save_dict = {}
count = 0
for law_article in tqdm(data):
    law_id = law_article["law_id"]
    law_articles = law_article["articles"]
    
    for sub_article in law_articles:
        article_id = sub_article["article_id"]
        article_title = sub_article["title"]
        article_text = sub_article["text"]
        article_full = article_title + ". " + article_text
        article_full = article_full.replace("\n", " ")
        concat_id = law_id + "_" + article_id
        if concat_id not in save_dict:
            count += 1
            save_dict[concat_id] = {"title": article_title, "text": article_text}
print(count)
print("Create legal dict from raw data")
with open(os.path.join(dest_path, "legal_dict.json"), "w") as outfile:
        json.dump(save_dict, outfile)

100%|██████████| 3271/3271 [00:00<00:00, 15988.98it/s]


61425
Create legal dict from raw data


In [ ]:
print("Process documents")
documents = []
doc_refers = []
for law_article in tqdm(data):
    law_id = law_article["law_id"]
    law_articles = law_article["articles"]
    
    for sub_article in law_articles:
        article_id = sub_article["article_id"]
        article_title = sub_article["title"]
        article_text = sub_article["text"]
        article_full = article_title + " " + article_text
            
        tokens = bm25_tokenizer(article_full)
        documents.append(tokens)
        doc_refers.append([law_id, article_id, article_full])

Process documents


100%|██████████| 3271/3271 [18:15<00:00,  2.99it/s]


In [ ]:
# Lưu file token và văn bản pháp luật
dest_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data'
with open(os.path.join(dest_path, "documents_bm25token"), "wb") as documents_file:
    pickle.dump(documents, documents_file)
with open(os.path.join(dest_path,"doc_refers_saved"), "wb") as doc_refer_file:
    pickle.dump(doc_refers, doc_refer_file)

In [ ]:
# Đọc file
dest_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data'
with open(os.path.join(dest_path, "documents_bm25token"), "rb") as documents_file:
    documents = pickle.load(documents_file)
with open(os.path.join(dest_path,"doc_refers_saved"), "rb") as doc_refer_file:
    doc_refers = pickle.load(doc_refer_file)

In [ ]:
# Khởi tạo BM25 với token thông thường
# Setup tham số BM25
bm25_k1 = 0.4
bm25_b = 0.6
bm25 = BM25Plus(documents, k1=bm25_k1, b=bm25_b)
# Lưu model
with open(os.path.join(model_path, "bm25_Plus_with_tokenizer_stopword_manual"), "wb") as bm_file:
    pickle.dump(bm25, bm_file)

### Đánh giá Training question với top 5


In [ ]:
# Đánh giá Training question với top 5
top_k_bm25 = 5
total_f2 = 0
total_precision = 0
total_recall = 0
questions_train_path = os.path.join(dest_path, "questions_train.json")
with open(questions_train_path) as json_file:
    questions_train = json.load(json_file)
for idx, item in tqdm(enumerate(questions_train)):
    question_id = item["question_id"]
    question = item["question"]
    relevant_articles = item["relevant_articles"]
    actual_positive = len(relevant_articles)
    # Tokenizer câu query
    tokenized_query = bm25_tokenizer(question)
    doc_scores = bm25.get_scores(tokenized_query)
    # Lấy top N. N lớn: giảm precision, tăng recall. N nhỏ: tăng precision, giảm recall
    # argpartition sắp xếp lại
    predictions = np.argpartition(doc_scores, len(doc_scores) - top_k_bm25)[-top_k_bm25:]
    # Trick to balance precision and recall
    if doc_scores[predictions[1]] - doc_scores[predictions[0]] >= 2.7:
        predictions = [predictions[1]]
    true_positive = 0
    false_positive = 0
    for idx, idx_pred in enumerate(predictions):
        pred = doc_refers[idx_pred]
        # Remove prediction with too low score: 20
        if doc_scores[idx_pred] >= 20:
            for article in relevant_articles:
                if pred[0] == article["law_id"] and pred[1] == article["article_id"]:
                    true_positive += 1
                else:
                    false_positive += 1
    precision = true_positive/(true_positive + false_positive + 1e-20)
    recall = true_positive/actual_positive
    f2 = calculate_f2(precision, recall)
    total_precision += precision
    total_recall += recall
    total_f2 += f2

2716it [12:58,  3.49it/s]


In [ ]:
k = len(questions_train)
print(f"Average F2: \t\t\t\t{total_f2/k}")
print(f"Average Precision: {total_precision/k}")
print(f"Average Recall: {total_recall/k}\n")

Average F2: 				0.3869371488572473
Average Precision: 0.15870765832105574
Average Recall: 0.6721894943544428



In [ ]:
# Đánh giá Test question với top 5
top_k_bm25 = 50
total_f2 = 0
total_precision = 0
total_recall = 0
questions_test_path = os.path.join(dest_path, "questions_test.json")
with open(questions_test_path) as json_file:
    questions_test = json.load(json_file)
for idx, item in tqdm(enumerate(questions_test)):
    question_id = item["question_id"]
    question = item["question"]
    relevant_articles = item["relevant_articles"]
    actual_positive = len(relevant_articles)
    # Tokenizer câu query
    tokenized_query = bm25_tokenizer(question)
    doc_scores = bm25.get_scores(tokenized_query)
    # Lấy top N. N lớn: giảm precision, tăng recall. N nhỏ: tăng precision, giảm recall
    # argpartition sắp xếp lại
    predictions = np.argpartition(doc_scores, len(doc_scores) - top_k_bm25)[-top_k_bm25:]
    # Trick to balance precision and recall
    # if doc_scores[predictions[1]] - doc_scores[predictions[0]] >= 2.7:
    #     predictions = [predictions[1]]
    true_positive = 0
    false_positive = 0
    for idx, idx_pred in enumerate(predictions):
        pred = doc_refers[idx_pred]
        # Remove prediction with too low score: 20
        # if doc_scores[idx_pred] >= 20:
        for article in relevant_articles:
            if pred[0] == article["law_id"] and pred[1] == article["article_id"]:
                true_positive += 1
            else:
                false_positive += 1
    precision = true_positive/(true_positive + false_positive + 1e-20)
    recall = true_positive/actual_positive
    f2 = calculate_f2(precision, recall)
    total_precision += precision
    total_recall += recall
    total_f2 += f2

480it [02:17,  3.49it/s]


In [ ]:
k = len(questions_test)
print(f"Average F2: \t\t\t\t{total_f2/k}")
print(f"Average Precision: {total_precision/k}")
print(f"Average Recall: {total_recall/k}\n")

Average F2: 				0.08339763374485626
Average Precision: 0.018013888888888687
Average Recall: 0.9006944444444444



## Tạo các cặp Negative pair từ BM25 với top20 và top50


In [ ]:
with open(os.path.join(model_path, "bm25_Plus_with_tokenizer_stopword_manual"), "rb") as bm_file:
    bm25 = pickle.load(bm_file)
with open(os.path.join(dest_path,"doc_refers_saved"), "rb") as doc_refer_file:
    doc_refers = pickle.load(doc_refer_file)

In [ ]:
save_pairs = []
top_k = 50 #20
questions_train_path = os.path.join(dest_path, "questions_train.json")
with open(questions_train_path) as json_file:
    questions_train = json.load(json_file)
doc_data = json.load(open(os.path.join(dest_path, "legal_dict.json")))

for idx, item in tqdm(enumerate(questions_train)):
      question_id = item["question_id"]
      question = item["question"]
      relevant_articles = item["relevant_articles"]
      actual_positive = len(relevant_articles)
      
      tokenized_query = bm25_tokenizer(question)
      doc_scores = bm25.get_scores(tokenized_query)

      predictions = np.argpartition(doc_scores, len(doc_scores) - top_k)[-top_k:]
      # Save positive pairs
      for article in relevant_articles:
          save_dict = {}
          save_dict["question"] = question
          concat_id = article["law_id"] + "_" + article["article_id"]
          save_dict["document"] = doc_data[concat_id]["title"] + " " + doc_data[concat_id]["text"]
          save_dict["relevant"] = 1
          save_pairs.append(save_dict)

      # Save negative pairs
      for idx, idx_pred in enumerate(predictions):
          pred = doc_refers[idx_pred]

          check = 0
          for article in relevant_articles:
              if pred[0] == article["law_id"] and pred[1] == article["article_id"]:
                  check += 1

          if check == 0:
              save_dict = {}
              save_dict["question"] = question
              concat_id = pred[0] + "_" + pred[1]
              save_dict["document"] = doc_data[concat_id]["title"] + " " + doc_data[concat_id]["text"]
              save_dict["relevant"] = 0
              save_pairs.append(save_dict)

2716it [14:44,  3.07it/s]


In [ ]:
#  with open(os.path.join(dest_path, f"bm_25_pairs_top{top_k}"), "wb") as pair_file:
#         pickle.dump(save_pairs, pair_file)
# print(len(save_pairs))

54697


In [ ]:
 with open(os.path.join(dest_path, f"bm_25_pairs_top{top_k}"), "wb") as pair_file:
        pickle.dump(save_pairs, pair_file)
print(len(save_pairs))

136024


## Train Negative pair với Sentence Transformer

In [3]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 4.0 MB/s 
     |████████████████████████████████| 4.2 MB 14.2 MB/s 
     |████████████████████████████████| 1.2 MB 50.2 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
     |████████████████████████████████| 6.6 MB 63.3 MB/s 
     |████████████████████████████████| 596 kB 56.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=61f6515290a6a95241041a0772b05960e08ff88db082c6b122fdb3b619906492
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from sentence_transformers import SentenceTransformer, models, InputExample, losses, LoggingHandler
from torch.utils.data import DataLoader
import pickle
from sentence_transformers import evaluation
import logging
import argparse
import os

In [5]:
# Tham số
top_k = 20
max_seq_length = 256
# tập validation 
num_val = 50*50
# fit training
epochs = 5
batch_size = 32
lr = 1e-5
model_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Model'
dest_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data'


In [6]:
word_embedding_model = models.Transformer("vinai/phobert-base", max_seq_length=max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
print(model)

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [7]:
with open("/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data/bm_25_pairs_top20", "rb") as pair_file:
    pairs = pickle.load(pair_file)
print(f"There are {len(pairs)} pair sentences.")
train_examples = []
sent1 = []
sent2 = []
scores = []
num_train = len(pairs) - num_val
print(num_train)


There are 54697 pair sentences.
52197


In [8]:
for idx, pair in enumerate(pairs):
    relevant = float(pair["relevant"])
    question = pair["question"]
    document = pair["document"]
    if idx <= num_train:
        example = InputExample(texts=[question, document], label=relevant)
        train_examples.append(example)
    else:
        sent1.append(question)
        sent2.append(document)
        scores.append(relevant)

print("Number of sample for training: ", len(train_examples))

Number of sample for training:  52198


In [9]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
train_loss = losses.ContrastiveLoss(model)
evaluator = evaluation.BinaryClassificationEvaluator(sent1, sent2, scores)

In [11]:
output_path = os.path.join(dest_path, "pho_bert_base_with_pair_top_20_constractive_loss")
os.makedirs(output_path, exist_ok=True)
model.fit(train_objectives=[(train_dataloader, train_loss)], 
            epochs=epochs, 
            warmup_steps=1000,
            optimizer_params={'lr': 1e-5},
            save_best_model=True,
            evaluator=evaluator,
            evaluation_steps=num_val,
            output_path=output_path,
            use_amp=True,
            show_progress_bar=True)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1632 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1632 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1632 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1632 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1632 [00:00<?, ?it/s]

In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

## Thử nghiệm sinh ra answer với question và context

In [ ]:
from transformers import pipeline
# model_checkpoint = "nguyenvulebinh/vi-mrc-large"
model_checkpoint = "nguyenvulebinh/vi-mrc-base"
nlp = pipeline('question-answering', model=model_checkpoint,
                   tokenizer=model_checkpoint)
QA_input = {
  'question': "Bình là chuyên gia về gì ?",
  'context': "Bình Nguyễn là một người đam mê với lĩnh vực xử lý ngôn ngữ tự nhiên . Anh nhận chứng chỉ Google Developer Expert năm 2020"
}
res = nlp(QA_input)
print('pipeline: {}'.format(res))

Downloading:   0%|          | 0.00/688 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

pipeline: {'score': 0.578204870223999, 'start': 45, 'end': 68, 'answer': 'xử lý ngôn ngữ tự nhiên'}


In [ ]:
import os
import json
from tqdm import tqdm

dest_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data'

questions_test_path = os.path.join(dest_path, "questions_test.json")
with open(questions_test_path) as json_file:
    questions_test = json.load(json_file)

In [ ]:
legal_corpus_path = "/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/zac2021-ltr-data/legal_corpus.json"
with open(legal_corpus_path) as json_file:
    data = json.load(json_file)
print(data[0])
print(type(data))

{'law_id': '01/2009/tt-bnn', 'articles': [{'article_id': '1', 'title': 'Điều 1. Phạm vi áp dụng', 'text': 'Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.'}, {'article_id': '2', 'title': 'Điều 2. Tổ chức lực lượng', 'text': '1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân cấp xã nơi có đê phải tổ chức lực lượng lao động tại địa phương để tuần tra, canh gác đê và thường trực trên các điếm canh đê hoặc nhà dân khu vực gần đê (đối với những khu vực chưa có điếm canh đê), khi có báo động lũ từ cấp I trở lên đối với tuyến sông có đê (sau đây gọi tắt là lực lượng tuần tra, canh gác đê).\n2. Lực lượng tuần tra, canh gác đê được tổ chức thành các đội, do Ủy ban nhân dân cấp xã ra quyết định thành lập; từ 01 đến 02 kilômét đê thành lập 01 đội; mỗi đội có từ 12 đến 18 người, trong đó có 01 đội trưởng và 01 hoặc 02 đội phó. Danh sách thành viên đội tuần tra, canh gác đê được niêm yết

In [ ]:
for law_article in tqdm(data):
    law_id = law_article["law_id"]
    if law_id == questions_test[0]['relevant_articles'][0]['law_id']:
      law_articles = law_article["articles"]
      break
for sub_article in law_articles:
  article_id = sub_article["article_id"]
  if article_id == questions_test[0]['relevant_articles'][0]['article_id']:
    article_title = sub_article["title"]
    article_text = sub_article["text"]
    article_full = article_title + ". " + article_text
    article_full = article_full.replace("\n", " ")
    break
context = article_full

 93%|█████████▎| 3034/3271 [00:00<00:00, 832603.92it/s]


In [ ]:
print(context)

Điều 8. Cung cấp dịch vụ công trực tuyến. 1. Dịch vụ công trực tuyến phục vụ cấp, quản lý, kiểm soát hộ chiếu của công dân Việt Nam được triển khai trên Cổng dịch vụ công của Bộ Công an, Bộ Ngoại giao theo chức năng, nhiệm vụ được phân công và liên kết, tích hợp thông tin với cổng dịch vụ công quốc gia. 2. Dịch vụ công trực tuyến nêu tại khoản 1 Điều này cho phép tiếp nhận thông tin khai tờ khai đề nghị cấp hộ chiếu, đặt lịch hẹn và hỗ trợ cung cấp thông tin hướng dẫn trình tự thực hiện cấp hộ chiếu; tra cứu trạng thái, kết quả xử lý đề nghị cấp hộ chiếu.


In [ ]:
context = data['law_id' == '77/2020/nđ-cp']['articles']['article_id' == '8']
print(context)

{'article_id': '1', 'title': 'Điều 1. Phạm vi áp dụng', 'text': 'Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.'}


In [ ]:
questions_test[0]

{'question': 'Những dịch vụ nào được cung cấp trực tuyến trong việc kiểm soát hộ chiếu công dân Việt Nam?',
 'question_id': '142e0483fd375f557561ace15e2aea83',
 'relevant_articles': [{'article_id': '8', 'law_id': '77/2020/nđ-cp'}]}

In [ ]:
question = questions_test[100]
for law_article in tqdm(data):
    law_id = law_article["law_id"]
    if law_id == question['relevant_articles'][0]['law_id']:
      law_articles = law_article["articles"]
      break
for sub_article in law_articles:
  article_id = sub_article["article_id"]
  if article_id == question['relevant_articles'][0]['article_id']:
    article_title = sub_article["title"]
    article_text = sub_article["text"]
    article_full = article_title + ". " + article_text
    article_full = article_full.replace("\n", " ")
    break
context = article_full
QA_input = {
  'question': question['question'],
  'context': context
}

print(QA_input)
res = nlp(QA_input)
print('pipeline: {}'.format(res))

 19%|█▉        | 618/3271 [00:00<00:00, 365699.76it/s]
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:300: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)


{'question': 'Quay đầu xe trên cầu thì bị phạt bao nhiêu?', 'context': 'Điều 5. Xử phạt người điều khiển xe ô tô và các loại xe tương tự xe ô tô vi phạm quy tắc giao thông đường bộ. 1. Phạt tiền từ 200.000 đồng đến 400.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây: a) Không chấp hành hiệu lệnh, chỉ dẫn của biển báo hiệu, vạch kẻ đường, trừ các hành vi vi phạm quy định tại điểm a, điểm d, điểm g, điểm h, điểm i, điểm k khoản 2; điểm a, điểm d, điểm đ, điểm e, điểm k, điểm l, điểm o, điểm r, điểm s khoản 3; điểm b, điểm c, điểm d, điểm đ, điểm I khoản 4; điểm a, điểm c, điểm d, điểm đ, điểm e, điểm g, điểm i khoản 5; điểm a, điểm b khoản 6; điểm a, điểm c khoản 7; điểm a khoản 8 Điều này; b) Chuyển hướng không nhường quyền đi trước cho: Người đi bộ, xe lăn của người khuyết tật qua đường tại nơi có vạch kẻ đường dành cho người đi bộ; xe thô sơ đang đi trên phần đường dành cho xe thô sơ; c) Chuyển hướng không nhường đường cho: Các xe đi ngược chiều; n

## FAISS


In [ ]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 4.0 MB/s 


### Thử nghiệm

In [ ]:
import requests
from io import StringIO
import pandas as pd

In [ ]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [ ]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [ ]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [ ]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [ ]:
# each of these dataset have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 191: expected 3 fields, saw 4\nSkipping line 206: expected 3 fields, saw 4\nSkipping line 295: expected 3 fields, saw 4\nSkipping line 695: expected 3 fields, saw 4\nSkipping line 699: expected 3 fields, saw 4\n'
b'Skipping line 104: expected 3 fields, saw 4\nSkipping line 181: expected 3 fields, saw 4\nSkipping line 317: expected 3 fields, saw 4\nSkipping line 412: expected 3 fields, saw 5\nSkipping line 508: expected 3 fields, saw 4\n'


In [ ]:
len(set(sentences))


4802

In [ ]:

# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [ ]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(4802, 768)

In [ ]:
type(sentence_embeddings)

numpy.ndarray

In [ ]:
import faiss


In [ ]:
d = sentence_embeddings.shape[1]
d

768

In [ ]:
index = faiss.IndexFlatL2(d)
index.is_trained

True

In [ ]:
index.add(sentence_embeddings)
index.ntotal

4802

In [ ]:
%%time
k = 4
xq = model.encode(["Someone sprints with a football"])
D, I = index.search(xq, k)  # search
print(I)

[[ 374 1364 2060   13]]
CPU times: user 7.41 ms, sys: 0 ns, total: 7.41 ms
Wall time: 6.7 ms


In [ ]:
data['sentence_A'].iloc[[ 374, 1364, 2060 ,  13]]

374     The ice skating rink placed outdoors is full o...
1364                           A person is brushing a cat
2060                  An onion is being sliced by a woman
13      The player is dunking the basketball into the ...
Name: sentence_A, dtype: object

### Ứng dụng

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 4.7 MB/s 
     |████████████████████████████████| 4.2 MB 18.0 MB/s 
     |████████████████████████████████| 1.2 MB 48.1 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 6.6 MB 35.6 MB/s 
     |████████████████████████████████| 596 kB 49.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=e56ae01e9ec2851a1fc746f8b0cdf18a9f920fdc86d5f6caaf1a28b24c6ea879
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import faiss
import pickle
from sentence_transformers import SentenceTransformer, models, InputExample, losses, LoggingHandler
from torch.utils.data import DataLoader
import pickle
from sentence_transformers import evaluation
import logging
import argparse
import os

In [ ]:
# Đọc file
dest_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data'
with open(os.path.join(dest_path, "documents_bm25token"), "rb") as documents_file:
    documents = pickle.load(documents_file)
with open(os.path.join(dest_path,"doc_refers_saved"), "rb") as doc_refer_file:
    doc_refers = pickle.load(doc_refer_file)

In [ ]:
doc_refers[0]

['01/2009/tt-bnn',
 '1',
 'Điều 1. Phạm vi áp dụng Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.']

In [ ]:
doc_refers[1]

['01/2009/tt-bnn',
 '2',
 'Điều 2. Tổ chức lực lượng 1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân cấp xã nơi có đê phải tổ chức lực lượng lao động tại địa phương để tuần tra, canh gác đê và thường trực trên các điếm canh đê hoặc nhà dân khu vực gần đê (đối với những khu vực chưa có điếm canh đê), khi có báo động lũ từ cấp I trở lên đối với tuyến sông có đê (sau đây gọi tắt là lực lượng tuần tra, canh gác đê).\n2. Lực lượng tuần tra, canh gác đê được tổ chức thành các đội, do Ủy ban nhân dân cấp xã ra quyết định thành lập; từ 01 đến 02 kilômét đê thành lập 01 đội; mỗi đội có từ 12 đến 18 người, trong đó có 01 đội trưởng và 01 hoặc 02 đội phó. Danh sách thành viên đội tuần tra, canh gác đê được niêm yết tại điếm canh đê thuộc địa bàn được phân công.\n3. Khi lũ, bão có diễn biến phức tạp, kéo dài ngày, Uỷ ban nhân dân cấp xã có thể quyết định việc bổ sung thêm thành viên cho đội tuần tra, canh gác đê.']

In [ ]:
sentences = []
for item in doc_refers:
  sentences.append(item[2])
len(set(sentences))

60703

In [ ]:
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [ ]:
len(set(sentences))

60703

In [ ]:
print(sentences[:20])

['Điều 15. Phối hợp với các khoa hệ ngoại 1. Các khoa thuộc hệ ngoại có trách nhiệm:\n- Lập hồ sơ bệnh án, thăm khám và làm các xét nghiệm hoặc thủ thuật cần thiết để chẩn đoán và chỉ định phương pháp phẫu thuật.\n- Tổ chức đưa người bệnh dự kiến phẫu thuật theo kế hoạch tới khoa gây mê - hồi sức để khám trước mổ. Trường hợp người bệnh nặng hoặc tình huống cấp cứu thì thông báo để khoa gây mê - hồi sức cử bác sỹ đến khám.\n- Lập kế hoạch phẫu thuật và thông báo với khoa gây mê - hồi sức đầy đủ các thông tin: họ tên người bệnh, tuổi, giới, số buồng bệnh, chẩn đoán, dự kiến phương pháp và thời gian phẫu thuật, họ tên các phẫu thuật viên, mã số hồ sơ theo dõi (nếu quản lý trên hệ thống máy tính).\n- Bác sĩ phẫu thuật có trách nhiệm theo dõi, thăm khám, phối hợp thực hiện các phác đồ điều trị và xử trí những bất thường liên quan đến phẫu thuật đã thực hiện trên người bệnh.\n2. Khoa gây mê - hồi sức có trách nhiệm:\n- Khám người bệnh trước khi phẫu thuật, ra y lệnh điều trị bổ sung trước kh

In [ ]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

(60703, 768)

In [ ]:
import numpy as np
np.savetxt('/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data/token_corpus_faiss_bert-base-nli-mean-tokens.out', sentence_embeddings)   # X is an array

In [ ]:
import faiss
d = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.is_trained

True

In [ ]:
index.add(sentence_embeddings)
index.ntotal

60703

In [ ]:
%%time
k = 4
xq = model.encode(["Những dịch vụ nào được cung cấp trực tuyến trong việc kiểm soát hộ chiếu công dân Việt Nam?"])
D, I = index.search(xq, k)  # search
print(I)

[[  626 15794 20305  6898]]
CPU times: user 621 ms, sys: 6 ms, total: 627 ms
Wall time: 647 ms


In [ ]:
#data['sentence_A'].iloc[[ 374, 1364, 2060 ,  13]]
for i in I: 
  for x in i:
    print(doc_refers[x])
    print(sentences[x])

['01/2015/tt-nhnn', '18', 'Điều 18. Báo cáo Hàng tháng, chậm nhất vào ngày 12 tháng liền kề sau tháng báo cáo, tổ chức tín dụng, chi nhánh ngân hàng nước ngoài báo cáo Ngân hàng Nhà nước Việt Nam (Vụ Chính sách tiền tệ) về hoạt động kinh doanh, cung ứng và sử dụng sản phẩm phái sinh lãi suất theo các biểu số 01 và 02 ban hành kèm theo Thông này.']
Điều 1. Phạm vi điều chỉnh Thông tư này quy định quy trình tuần tra, kiểm tra, kiểm soát của Cảnh sát biển Việt Nam.
['11/2021/nđ-cp', '27', 'Điều 27. Nội dung thẩm định hồ sơ đề nghị giao, công nhận, trả lại khu vực biển, gia hạn, sửa đổi, bổ sung Quyết định giao khu vực biển 1. Nội dung thẩm định hồ sơ đề nghị giao khu vực biển:\na) Tính đầy đủ về hình thức và nội dung của hồ sơ;\nb) Sự phù hợp của khu vực biển đề nghị giao với quy hoạch quy định tại khoản 3 Điều 5 Nghị định này; trường hợp chưa có quy hoạch phải bảo đảm phù hợp với nội dung quy định tại các điểm a, b và điểm c khoản 4 Điều 5 Nghị định này;\nc) Sự phù hợp của hoạt động khai

### Thử nghiệm 1 model phobert khác

In [ ]:
from sentence_transformers import SentenceTransformer

model_embedding = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

def embed_text(batch_text):
    batch_embedding = model_embedding.encode(batch_text)
    return [vector.tolist() for vector in batch_embedding]


Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/VoVanPhuc_sup-SimCSE-VietNamese-phobert-base were not used when initializing RobertaModel: ['mlp.dense.weight', 'mlp.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyvi.ViTokenizer import tokenize
import pandas as pd
def index_batch(sentences):
    requests = []
    titles = [tokenize(sentence) for sentence in sentences]
    embedding = embed_text(titles)
    return embedding
sentence_embeddings = index_batch(sentences)
sentence_embeddings.shape

AttributeError: ignored

In [ ]:
len(sentence_embeddings)

60703

In [ ]:
import numpy as np
np.savetxt('/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data/token_corpus_faiss_sup-SimCSE-VietNamese-phobert-base.out', sentence_embeddings)   # X is an array

In [ ]:
import faiss
d = sentence_embeddings_all.shape[1]
index = faiss.IndexFlatL2(d)
index.is_trained

True

In [ ]:
index.add(sentence_embeddings_all)
index.ntotal

60703

In [ ]:
%%time
k = 4
xq = model.encode(["Những dịch vụ nào được cung cấp trực tuyến trong việc kiểm soát hộ chiếu công dân Việt Nam?"])
D, I = index.search(xq, k)  # search
print(I)

[[43979 31663 26149 11020]]
CPU times: user 76 ms, sys: 999 µs, total: 77 ms
Wall time: 106 ms


In [ ]:
#data['sentence_A'].iloc[[ 374, 1364, 2060 ,  13]]
for i in I: 
  for x in i:
    print(doc_refers[x])
    print(sentences[x])

['43/2018/nđ-cp', '13', 'Điều 13. Cơ quan được giao quản lý tài sản trực tiếp tổ chức khai thác tài sản kết cấu hạ tầng hàng hải 1. Cơ quan được giao quản lý tài sản trực tiếp tổ chức khai thác tài sản trong các trường hợp sau:\na) Tài sản kết cấu hạ tầng hàng hải liên quan đến an ninh quốc gia;\nb) Áp dụng phương thức trực tiếp khai thác hiệu quả hơn phương thức quy định tại các điểm b, điểm c khoản 1 Điều 12 Nghị định này;\nc) Không có tổ chức đăng ký thực hiện phương thức quy định tại các điểm b, điểm c khoản 1 Điều 12 Nghị định này.\n2. Thẩm quyền phê duyệt Đề án khai thác tài sản kết cấu hạ tầng hàng hải:\na) Thủ tướng Chính phủ phê duyệt Đề án khai thác tài sản kết cấu hạ tầng hàng hải liên quan đến quốc phòng, an ninh quốc gia, tài sản hình thành từ dự án do Thủ tướng Chính phủ quyết định đầu tư theo quy định trên cơ sở đề nghị của Bộ trưởng Bộ Giao thông vận tải, ý kiến của Bộ Tài chính và các cơ quan có liên quan;\nb) Bộ trưởng Bộ Giao thông vận tải phê duyệt Đề án khai thác t

In [ ]:
# Calculate length of maximal list
n = len(max(sentence_embeddings, key=len))
# Make the lists equal in length
sentence_embeddings_2 = [x + [0]*(n-len(x)) for x in sentence_embeddings]

In [ ]:
import numpy as np
sentence_embeddings_all = np.array(sentence_embeddings_2, dtype=np.float32)

In [ ]:
sentence_embeddings_all.shape

(60703, 768)

In [ ]:
import numpy
# converting list to array
arr = numpy.array(sentence_embeddings)
arr.shape

In [ ]:
d = arr.shape[1]
d

60703

In [ ]:
index = faiss.IndexFlatL2(d)
index.is_trained

True

In [ ]:
k = 4
questions_train_path = os.path.join(dest_path, "questions_train.json")
with open(questions_train_path) as json_file:
    questions_train = json.load(json_file)

xq = model.encode(["Someone sprints with a football"])

In [ ]:
%%time
# Đánh giá Training question với top 5
top_k_bm25 = 5
total_f2 = 0
total_precision = 0
total_recall = 0
questions_train_path = os.path.join(dest_path, "questions_train.json")
with open(questions_train_path) as json_file:
    questions_train = json.load(json_file)
for idx, item in tqdm(enumerate(questions_train[:1])):
    question_id = item["question_id"]
    question = item["question"]
    relevant_articles = item["relevant_articles"]
    actual_positive = len(relevant_articles)
    # Tokenizer câu query
    tokenized_query = bm25_tokenizer(question)
    D, I = index.search(xq, k)  # search
    print(I)


In [ ]:
doc_refers.iloc[[4586, 10252, 12465, 190]]


# Predict


In [11]:
!pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 5.2 MB/s 
     |████████████████████████████████| 4.2 MB 17.3 MB/s 
     |████████████████████████████████| 1.2 MB 62.4 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 6.6 MB 59.3 MB/s 
     |████████████████████████████████| 596 kB 61.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=a1436c74b3ae0761676804d7dd228c582c361538bebd78b60d40081ff8a723e8
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [40]:
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 6.9 MB/s 
     |████████████████████████████████| 965 kB 59.6 MB/s 
     |████████████████████████████████| 235 kB 70.7 MB/s 
     |████████████████████████████████| 581 kB 87.2 MB/s 


In [18]:
import numpy as np
import json
import torch
from tqdm import tqdm
from rank_bm25 import *
import argparse
import os
import pickle
import glob
from sentence_transformers import SentenceTransformer, util

In [1]:
def encode_legal_data(legal_dict_json, models):
    # print(legal_dict_json)
    doc_data = json.load(open(legal_dict_json))
    # print(len(doc_data))
    list_emb_models = []
    for model in models:
        emb2_list = []
        for k, doc in tqdm(doc_data.items()):
            emb2 = model.encode(doc_data[k]["title"] + " " + doc_data[k]["text"])
            emb2_list.append(emb2)
        emb2_arr = np.array(emb2_list)
        list_emb_models.append(emb2_arr)
    return list_emb_models

In [2]:
def encode_question(question_data, models):
    print("Start encoding questions.")
    question_embs = []
    for model in models:
        emb_quest_dict = {}
        for _, item in tqdm(enumerate(question_data)):
            question_id = item["question_id"]
            question = item["question"]
            emb_quest_dict[question_id] = model.encode(question)
        question_embs.append(emb_quest_dict)
    return question_embs

In [3]:
def load_encoded_legal_corpus(legal_data_path):
    print("Start loading legal corpus.")
    with open(legal_data_path, "rb") as f1:
        emb_legal_data = pickle.load(f1)
    return emb_legal_data

In [4]:
def load_bm25(bm25_path):
    with open(bm25_path, "rb") as bm_file:
        bm25 = pickle.load(bm_file)
    return bm25

In [14]:
def load_models(root, model_names):
    models = []
    for model_path in tqdm(model_names):
        model_path = os.path.join(dest_path, model_path)
        models.append(SentenceTransformer(model_path))
    return models

In [6]:
def load_question_json(question_path):
    question_path = glob.glob(f"{question_path}/*.json")[0]
    question_data = json.load(open(question_path))
    return question_data

## Load model


In [22]:
print("Start loading model.")
dest_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data'
model_name = ['pho_bert_base_with_pair_top_50_constractive_loss']
models = load_models(dest_path, model_name)
print("Number of pretrained models: ", len(models))

Start loading model.


100%|██████████| 1/1 [00:15<00:00, 15.57s/it]

Number of pretrained models:  1


In [23]:
# load question from json file
dest_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data'
questions_train_path = os.path.join(dest_path, "questions_train.json")
with open(questions_train_path) as json_file:
    questions_train = json.load(json_file)

In [62]:
# load question from json file
dest_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Training Data'
questions_test_path = os.path.join(dest_path, "questions_test.json")
with open(questions_test_path) as json_file:
    questions_test = json.load(json_file)

In [24]:
print("Number of questions: ", len(questions_train))

Number of questions:  2716


In [26]:
# load bm25 model 
model_path = '/content/drive/MyDrive/Tài liệu học kỳ/Học kỳ 20222/Đồ án tốt nghiệp/Model'
with open(os.path.join(model_path, "bm25_Plus_with_tokenizer_stopword_manual"), "rb") as bm_file:
    bm25 = pickle.load(bm_file)

In [27]:
# load corpus to search
print("Load legal data.")
with open(os.path.join(dest_path,"doc_refers_saved"), "rb") as doc_refer_file:
    doc_refers = pickle.load(doc_refer_file)

Load legal data.


In [29]:
# load pre encoded for legal corpus
if 1:
    emb_legal_data = encode_legal_data(dest_path + "/legal_dict.json", models)
else:
    emb_legal_data = load_encoded_legal_corpus('encoded_legal_data.pkl')

100%|██████████| 61425/61425 [14:11<00:00, 72.14it/s]


In [30]:
type(emb_legal_data)

list

In [34]:
with open(os.path.join(dest_path,"encoded_legal_data.pkl"),'wb') as f:
  pickle.dump(emb_legal_data, f)
 

In [35]:
# encode question for query
question_embs = encode_question(questions_train, models)

Start encoding questions.


2716it [00:26, 101.62it/s]


In [63]:
question_embs_test = encode_question(questions_test, models)

Start encoding questions.


480it [00:05, 92.34it/s] 


In [36]:
# define top n for compare and range of score
top_n = 61425
range_score = 2.6


In [43]:
# Tiến xử lý văn bản
from underthesea import word_tokenize
import string
def remove_stopword(w):
    stop_word = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10" ,
            "của", "và", "các", "có", "được", "theo", "tại", "trong", "về", 
            "hoặc", "người",  "này", "khoản", "cho", "không", "từ", "phải", 
            "ngày", "việc", "sau",  "để",  "đến", "bộ",  "với", "là", "năm", 
            "khi", "số", "trên", "khác", "đã", "thì", "thuộc", "điểm", "đồng",
            "do", "một", "bị", "vào", "lại", "ở", "nếu", "làm", "đây", 
            "như", "đó", "mà", "nơi", "”", "“"]
    return w not in stop_word

def remove_punctuation(w):
    return w not in string.punctuation

def lower_case(w):
    return w.lower()
# BM25 Tokenizer
def bm25_tokenizer(text):
    tokens = word_tokenize(text)
    tokens = list(map(lower_case, tokens))
    tokens = list(filter(remove_punctuation, tokens))
    tokens = list(filter(remove_stopword, tokens))
    return tokens

In [44]:
pred_list = []

print("Start calculating results.")
for idx, item in tqdm(enumerate(questions_train)):
    question_id = item["question_id"]
    question = item["question"]

    tokenized_query = bm25_tokenizer(question)
    doc_scores = bm25.get_scores(tokenized_query)

    weighted = [1] 
    cos_sim = []

    for idx_2, model in enumerate(models):
        emb1 = question_embs[idx_2][question_id]
        emb2 = emb_legal_data[idx_2]
        scores = util.cos_sim(emb1, emb2)
        cos_sim.append(weighted[idx_2] * scores)
    cos_sim = torch.cat(cos_sim, dim=0)

    cos_sim = torch.sum(cos_sim, dim=0).squeeze(0).numpy()
    new_scores = doc_scores * cos_sim
    max_score = np.max(new_scores)

    predictions = np.argpartition(new_scores, len(new_scores) - top_n)[-top_n:]
    new_scores = new_scores[predictions]

    new_predictions = np.where(new_scores >= (max_score - range_score))[0]
    map_ids = predictions[new_predictions]
    new_scores = new_scores[new_scores >= (max_score - range_score)]

    if new_scores.shape[0] > 5:
        predictions_2 = np.argpartition(new_scores, len(new_scores) - 5)[-5:]
        map_ids = map_ids[predictions_2]
        
    pred_dict = {}
    pred_dict["question_id"] = question_id
    pred_dict["relevant_articles"] = []

    # post processing character error
    dup_ans = []
    for idx, idx_pred in enumerate(map_ids):
        pred = doc_refers[idx_pred]
        law_id = pred[0]
        article_id = pred[1]
        
        if law_id.endswith("nd-cp"):
            law_id = law_id.replace("nd-cp", "nđ-cp")
        if law_id.endswith("nđ-"):
            law_id = law_id.replace("nđ-", "nđ-cp")
        if law_id.endswith("nð-cp"):
            law_id = law_id.replace("nð-cp", "nđ-cp")
        if law_id == "09/2014/ttlt-btp-tandtc-vksndtc":
            law_id = "09/2014/ttlt-btp-tandtc-vksndtc-btc"
        if law_id + "_" + article_id not in dup_ans:
            dup_ans.append(law_id + "_" + article_id)
            pred_dict["relevant_articles"].append({"law_id": law_id, "article_id": article_id})
    pred_list.append(pred_dict)

Start calculating results.


2716it [15:18,  2.96it/s]


In [46]:
# extract result
with open(os.path.join(dest_path,"submission_question_train.json"), 'w') as outfile:
    json.dump(pred_list, outfile)

In [51]:
print(pred_list[0])

{'question_id': '0637bf82c8b290c7875c5bfddbf91df5', 'relevant_articles': [{'law_id': '100/2019/nđ-cp', 'article_id': '37'}, {'law_id': '100/2019/nđ-cp', 'article_id': '82'}, {'law_id': '12/2010/tt-bca', 'article_id': '17'}, {'law_id': '47/2011/tt-bca', 'article_id': '7'}]}


In [50]:
questions_train[0]

{'question': 'Công an xã xử phạt lỗi không mang bằng lái xe có đúng không?',
 'question_id': '0637bf82c8b290c7875c5bfddbf91df5',
 'relevant_articles': [{'article_id': '7', 'law_id': '47/2011/tt-bca'}]}

In [58]:
# Chỉ số F2
def calculate_f2(precision, recall):        
    return (5 * precision * recall) / (4 * precision + recall + 1e-20)

In [60]:
true_positive = 0
false_positive = 0
actual_positive = len(questions_train)
for idx, predict in enumerate(pred_list):
    pred = predict['relevant_articles'][0]
    relevant_articles = questions_train[idx]['relevant_articles']
    # Remove prediction with too low score: 20
    # if doc_scores[idx_pred] >= 20:
    for article in relevant_articles:
        if pred["law_id"] == article["law_id"] and pred["article_id"] == article["article_id"]:
            true_positive += 1
        else:
            false_positive += 1
precision = true_positive/(true_positive + false_positive + 1e-20)
recall = true_positive/actual_positive
f2 = calculate_f2(precision, recall)

In [61]:
print(precision)
print(recall)
print(f2)

0.8332145402708482
0.8608247422680413
0.8551572787125092


In [64]:
pred_list = []

print("Start calculating results.")
for idx, item in tqdm(enumerate(questions_test)):
    question_id = item["question_id"]
    question = item["question"]

    tokenized_query = bm25_tokenizer(question)
    doc_scores = bm25.get_scores(tokenized_query)

    weighted = [1] 
    cos_sim = []

    for idx_2, model in enumerate(models):
        emb1 = question_embs_test[idx_2][question_id]
        emb2 = emb_legal_data[idx_2]
        scores = util.cos_sim(emb1, emb2)
        cos_sim.append(weighted[idx_2] * scores)
    cos_sim = torch.cat(cos_sim, dim=0)

    cos_sim = torch.sum(cos_sim, dim=0).squeeze(0).numpy()
    new_scores = doc_scores * cos_sim
    max_score = np.max(new_scores)

    predictions = np.argpartition(new_scores, len(new_scores) - top_n)[-top_n:]
    new_scores = new_scores[predictions]

    new_predictions = np.where(new_scores >= (max_score - range_score))[0]
    map_ids = predictions[new_predictions]
    new_scores = new_scores[new_scores >= (max_score - range_score)]

    if new_scores.shape[0] > 5:
        predictions_2 = np.argpartition(new_scores, len(new_scores) - 5)[-5:]
        map_ids = map_ids[predictions_2]
        
    pred_dict = {}
    pred_dict["question_id"] = question_id
    pred_dict["relevant_articles"] = []

    # post processing character error
    dup_ans = []
    for idx, idx_pred in enumerate(map_ids):
        pred = doc_refers[idx_pred]
        law_id = pred[0]
        article_id = pred[1]
        
        if law_id.endswith("nd-cp"):
            law_id = law_id.replace("nd-cp", "nđ-cp")
        if law_id.endswith("nđ-"):
            law_id = law_id.replace("nđ-", "nđ-cp")
        if law_id.endswith("nð-cp"):
            law_id = law_id.replace("nð-cp", "nđ-cp")
        if law_id == "09/2014/ttlt-btp-tandtc-vksndtc":
            law_id = "09/2014/ttlt-btp-tandtc-vksndtc-btc"
        if law_id + "_" + article_id not in dup_ans:
            dup_ans.append(law_id + "_" + article_id)
            pred_dict["relevant_articles"].append({"law_id": law_id, "article_id": article_id})
    pred_list.append(pred_dict)

Start calculating results.


480it [02:42,  2.95it/s]


In [70]:
true_positive = 0
false_positive = 0
actual_positive = len(questions_test)
for idx, predict in enumerate(pred_list):
    pred = predict['relevant_articles'][0]
    relevant_articles = questions_test[idx]['relevant_articles']
    # Remove prediction with too low score: 20
    # if doc_scores[idx_pred] >= 20:
    for article in relevant_articles:
        if pred["law_id"] == article["law_id"] and pred["article_id"] == article["article_id"]:
            true_positive += 1
        else:
            false_positive += 1
precision_test = true_positive/(true_positive + false_positive + 1e-20)
recall_test = true_positive/actual_positive
f2_test = calculate_f2(precision_test, recall_test)

In [71]:
print(precision_test)
print(recall_test)
print(f2_test)

0.6382113821138211
0.6541666666666667
0.6509121061359867
